In [ ]:
import tensorflow as tf
from tensorflow.python.keras.layers import (Flatten,LeakyReLU,AveragePooling2D,concatenate,
                                            Reshape,Activation, Conv2D, Input, MaxPooling2D,
                                            BatchNormalization, Lambda,SeparableConv2D,GlobalAveragePooling2D)
def resNetBlock(input_image):

  layer1 = SeparableConv2D(512, 3, strides=(1,1), padding='same', use_bias=False)(input_image)
  layer1 = BatchNormalization()(layer1)
  layer1 = LeakyReLU(alpha=0.1)(layer1)
  
  layer2 = SeparableConv2D(1024, 3, strides=(1,1), padding='same', use_bias=False)(layer1)
  layer2 = BatchNormalization()(layer2)
  layer2 = LeakyReLU(alpha=0.1)(layer2)
  
  layer3 = SeparableConv2D(256,3, strides=(1,1), padding='same', use_bias=False)(layer2)
  layer3 = BatchNormalization()(layer3)
  
  return layer3  

def build_model(img_size):
  input_shape = Input(shape=(img_size,img_size,3)) 

  layer_1 = SeparableConv2D(64, 3, strides=(1,1), padding='same', use_bias=False)(input_shape)
  layer_1 = BatchNormalization()(layer_1)
  layer_1 = LeakyReLU(alpha=0.1)(layer_1)
  
  layer_2 = SeparableConv2D(128, 3, strides=(1,1), padding='same', use_bias=False)(layer_1)
  layer_2 = BatchNormalization()(layer_2)
  layer_2 = LeakyReLU(alpha=0.1)(layer_2)
  
  layer_3 = SeparableConv2D(256, 3, strides=(1,1), padding='same', use_bias=False)(layer_2)
  layer_3 = BatchNormalization()(layer_3)
  layer_3 = LeakyReLU(alpha=0.1)(layer_3)
 
  resNetBlock1 = resNetBlock(layer_3)
  concact1 = concatenate([resNetBlock1, layer_3])
  layer_l = LeakyReLU(alpha=0.1)(concact1)
  maxpool1 = MaxPooling2D(pool_size=(2, 2))(layer_l)

  resNetBlock2 = resNetBlock(maxpool1)
  concact2 = concatenate([resNetBlock2, maxpool1])
  layer_l2 = LeakyReLU(alpha=0.1)(concact2)
  maxpool2 = MaxPooling2D(pool_size=(2, 2))(layer_l2)

  layer_z = SeparableConv2D(1024, 3, strides=(1,1), padding='same', use_bias=False)(maxpool1)
  layer_z = BatchNormalization()(layer_z)
  maxpool3 = MaxPooling2D(pool_size=(2, 2))(layer_z)

  resNetBlock3 = resNetBlock(maxpool2)
  concact3 = concatenate([resNetBlock3, maxpool2])
  concact4 = concatenate([concact3, maxpool3])
  layer_l3 = LeakyReLU(alpha=0.1)(concact4)

  layer_ant = Conv2D(200, (1,1), strides=(1,1), padding='same', use_bias=False)(layer_l3)
  global_avg = GlobalAveragePooling2D()(layer_ant)

  output = Activation('softmax')(global_avg)

  model = tf.keras.Model(inputs=input_shape, outputs=output)

  return model

In [ ]:
import os
import zipfile

ZIP_FILE = 'tiny-imagenet-200.zip'
INPUT_DATA_FOLDER= 'tiny-imagenet-200'

def extract_images():
    
    files = os.listdir('./')
    
    if ZIP_FILE not in files:
      os.system('wget http://cs231n.stanford.edu/tiny-imagenet-200.zip')
    if INPUT_DATA_FOLDER not in files:
      zip_ref = zipfile.ZipFile('tiny-imagenet-200.zip')
      zip_ref.extractall('./')
      zip_ref.close()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   brightness_range=(0.5,1.5),
                                   rotation_range=45,
                                   zoom_range=0.2,
                                   shear_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)

Using TensorFlow backend.


In [ ]:
import os

SAVED_MODEL = 'vddddAssignment4Amodel'

def fetch_model(input_size):
  files = os.listdir('/gdrive/My Drive/tpuweights/')

  matchs = [s for s in files if SAVED_MODEL in s]
  
  least_val_loss = 1000.00
  previous_model_file = ''
  for m in matchs:
      if float((m.split('_',2)[2]).split('.hdf5',1)[0]) < least_val_loss:
        least_val_loss = float((m.split('_',2)[2]).split('.hdf5',1)[0])
        previous_model_file = '/gdrive/My Drive/tpuweights/' + m
  if(least_val_loss < 1000.00):
      print("Loading previous model with least val loss: "+ str(previous_model_file))
      current_model = build_model(img_size=input_size)
      current_model.load_weights(previous_model_file)
  else:
      print("No previous model..   creating new Model")
      current_model = build_model(img_size=input_size)
  
  return current_model

In [ ]:
import pandas as pd

extract_images()

val_data = pd.read_csv('./tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)
val_data.head(3)

,File,Class
0,val_0.JPEG,n03444034
1,val_1.JPEG,n04067472
2,val_2.JPEG,n04070727


In [ ]:
from PIL import Image,ImageOps,ImageFilter
import numpy as np
from skimage.util import random_noise
import random

def additional_augmentation(batch):
  while True:
    batch_x, batch_y = next(batch)
    imageshape = batch_x.shape[1]
    final_batch_x_imgs = np.zeros((batch_x.shape[0], imageshape, imageshape, 3))
    for i in range(batch_x.shape[0]):
      pil_image = Image.fromarray((batch_x[i] * 255).astype('uint8'),mode='RGB')
      index = random.randint(0,25)
      if(index==0 or index > 14):
        final_batch_x_imgs[i] = batch_x[i]
        
      if(index==1 and imageshape >= 32):
        final_batch_x_imgs[i] = random_crops(pil_image,imageshape)
      if(index==2):
        final_batch_x_imgs[i] = randomnoise(batch_x[i],imageshape)
      if(index==3):
        final_batch_x_imgs[i] = autocontrast(pil_image,imageshape)
      if(index==4):
        final_batch_x_imgs[i] = equalize(pil_image,imageshape)
      if(index==5):
        final_batch_x_imgs[i] = blur(pil_image,imageshape)
      if(index==6):
        final_batch_x_imgs[i] = contour(pil_image,imageshape)
      if(index==7):
        final_batch_x_imgs[i] = detail(pil_image,imageshape)
      if(index==8):
        final_batch_x_imgs[i] = edge_enhance(pil_image,imageshape)
      if(index==9):
        final_batch_x_imgs[i] = edge_enhance_more(pil_image,imageshape)
      if(index==10):
        final_batch_x_imgs[i] = emboss(pil_image,imageshape)
      if(index==11):
        final_batch_x_imgs[i] = FIND_EDGES(pil_image,imageshape)
      if(index==12):
        final_batch_x_imgs[i] = SHARPEN(pil_image,imageshape)
      if(index==13):
        final_batch_x_imgs[i] = SMOOTH(pil_image,imageshape)
      if(index==14):
        final_batch_x_imgs[i] = SMOOTH_MORE(pil_image,imageshape)

      if(np.isnan(final_batch_x_imgs[i] ).any()):
        final_batch_x_imgs[i] = batch_x[i]
        
    yield (final_batch_x_imgs,batch_y)
    
# Resize Image 
def resize(image,currentsize):
  resized_img = np.zeros((currentsize, currentsize, 3))
  im2 = image.resize((currentsize, currentsize), Image.NEAREST)  
  resized_img = (np.array(im2.getdata()).reshape(currentsize, currentsize, 3))/255
  return resized_img

# Random Crop
def random_crops(image,currentsize):
  crop_size=0
  if(currentsize == 64):
    crop_size = 45
  if(currentsize == 32):
    crop_size = 25
  if(currentsize == 48):
    crop_size = 35
  height = np.random.randint(0, currentsize - crop_size + 1)
  im4 = image.crop((height,height, height+crop_size,height+crop_size))
  return resize(im4,currentsize)

# Random Noise
def randomnoise(image,currentsize):
  return random_noise(image)
   
#autocontrast
def autocontrast(image,currentsize):
  temp_img = np.zeros((currentsize, currentsize, 3))
  im2 = ImageOps.autocontrast(image)
  temp_img = (np.array(im2.getdata()).reshape(currentsize, currentsize, 3))/255
  return temp_img
  
#equalize
def equalize(image,currentsize):
  temp_img = np.zeros((currentsize, currentsize, 3))
  im2 = ImageOps.equalize(image)
  temp_img = (np.array(im2.getdata()).reshape(currentsize, currentsize, 3))/255
  return temp_img
  
#BLUR
def blur(image,currentsize):
  temp_img = np.zeros((currentsize, currentsize, 3))
  im2 = image.filter(ImageFilter.BLUR)
  temp_img = (np.array(im2.getdata()).reshape(currentsize, currentsize, 3))/255
  return temp_img

#CONTOUR
def contour(image,currentsize):
  temp_img = np.zeros((currentsize, currentsize, 3))
  im2 = image.filter(ImageFilter.CONTOUR)
  temp_img = (np.array(im2.getdata()).reshape(currentsize, currentsize, 3))/255
  return temp_img

#DETAIL
def detail(image,currentsize):
  temp_img = np.zeros((currentsize, currentsize, 3))
  im2 = image.filter(ImageFilter.DETAIL)
  temp_img = (np.array(im2.getdata()).reshape(currentsize, currentsize, 3))/255
  return temp_img

#EDGE_ENHANCE
def edge_enhance(image,currentsize):
  temp_img = np.zeros((currentsize, currentsize, 3))
  im2 = image.filter(ImageFilter.EDGE_ENHANCE)
  temp_img = (np.array(im2.getdata()).reshape(currentsize, currentsize, 3))/255
  return temp_img

#EDGE_ENHANCE_MORE
def edge_enhance_more(image,currentsize):
  temp_img = np.zeros((currentsize, currentsize, 3))
  im2 = image.filter(ImageFilter.EDGE_ENHANCE_MORE)
  temp_img = (np.array(im2.getdata()).reshape(currentsize, currentsize, 3))/255
  return temp_img

#EMBOSS
def emboss(image,currentsize):
  temp_img = np.zeros((currentsize, currentsize, 3))
  im2 = image.filter(ImageFilter.EMBOSS)
  temp_img = (np.array(im2.getdata()).reshape(currentsize, currentsize, 3))/255
  return temp_img

#FIND_EDGES
def FIND_EDGES(image,currentsize):
  temp_img = np.zeros((currentsize, currentsize, 3))
  im2 = image.filter(ImageFilter.FIND_EDGES)
  temp_img = (np.array(im2.getdata()).reshape(currentsize, currentsize, 3))/255
  return temp_img

#SHARPEN
def SHARPEN(image,currentsize):
  temp_img = np.zeros((currentsize, currentsize, 3))
  im2 = image.filter(ImageFilter.SHARPEN)
  temp_img = (np.array(im2.getdata()).reshape(currentsize, currentsize, 3))/255
  return temp_img

#SMOOTH
def SMOOTH(image,currentsize):
  temp_img = np.zeros((currentsize, currentsize, 3))
  im2 = image.filter(ImageFilter.SMOOTH)
  temp_img = (np.array(im2.getdata()).reshape(currentsize, currentsize, 3))/255
  return temp_img

#SMOOTH_MORE
def SMOOTH_MORE(image,currentsize):
  temp_img = np.zeros((currentsize, currentsize, 3))
  im2 = image.filter(ImageFilter.SMOOTH_MORE)
  temp_img = (np.array(im2.getdata()).reshape(currentsize, currentsize, 3))/255
  return temp_img

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
from tensorflow.python.keras.callbacks import ModelCheckpoint
import numpy as np

def learning_strategy(image_size,previous_epoch,epoch_size,batchsize,learning_rate):
  
  tf.keras.backend.clear_session()
  TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  
  modelx = fetch_model(image_size)
  modelx.compile(loss='categorical_crossentropy',optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate),metrics=['accuracy'])
  
  model_tpu = tf.contrib.tpu.keras_to_tpu_model(modelx,strategy=tf.contrib.tpu.TPUDistributionStrategy(tf.contrib.cluster_resolver.TPUClusterResolver(tpu=TPU_WORKER)))
  model_tpu.summary()
  
  train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(image_size,image_size),color_mode='rgb',batch_size=batchsize, class_mode='categorical', shuffle=True, seed=42)

  validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(image_size,image_size),
                                                    color_mode='rgb', class_mode='categorical', batch_size=batchsize, shuffle=True, seed=42)

  checkpoint = ModelCheckpoint('/gdrive/My Drive/tpuweights/vddddAssignment4Amodel_{epoch:02d}_{val_loss:.2f}.hdf5',monitor='val_acc',save_weights_only=True,save_best_only = True,mode='auto')
  model_tpu.fit_generator(additional_augmentation(train_generator),initial_epoch=previous_epoch,epochs=epoch_size,steps_per_epoch=int(100000/batchsize),verbose=1,validation_data=validation_generator,validation_steps=int(10000/batchsize),callbacks=[checkpoint])                       

In [ ]:
learning_strategy(image_size=32,previous_epoch=0,epoch_size=50,batchsize=1024,learning_rate=0.001)

[]
0
No previous model..   creating new Model
Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.83.166.114:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 13500332730615543303)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 10911548605046438221)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184

In [ ]:
learning_strategy(image_size=32,previous_epoch=50,epoch_size=100,batchsize=1024,learning_rate=0.0006)

Loading previous model with least val loss: /gdrive/My Drive/tpuweights/vddddAssignment4Amodel_47_2.54.hdf5
INFO:tensorflow:Querying Tensorflow master (grpc://10.83.166.114:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 13500332730615543303)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 10911548605046438221)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 13310310715681238334)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 13603759284196199019)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:

In [ ]:
learning_strategy(image_size=48,previous_epoch=100,epoch_size=150,batchsize=1024,learning_rate=0.0006)

Loading previous model with least val loss: /gdrive/My Drive/tpuweights/vddddAssignment4Amodel_51_2.38.hdf5
INFO:tensorflow:Querying Tensorflow master (grpc://10.83.166.114:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 13500332730615543303)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 10911548605046438221)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 13310310715681238334)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 13603759284196199019)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:

In [ ]:
learning_strategy(image_size=64,previous_epoch=150,epoch_size=200,batchsize=512,learning_rate=0.0003)

Loading previous model with least val loss: /gdrive/My Drive/tpuweights/vddddAssignment4Amodel_113_2.20.hdf5
Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.43.40.34:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 11008659628945883540)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5181798764399224991)
INFO:tensorflow:*** Available Device: _DeviceAttributes

In [ ]:
learning_strategy(image_size=64,previous_epoch=200,epoch_size=250,batchsize=512,learning_rate=0.0001)

Loading previous model with least val loss: /gdrive/My Drive/tpuweights/vddddAssignment4Amodel_167_2.00.hdf5
INFO:tensorflow:Querying Tensorflow master (grpc://10.43.40.34:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 11008659628945883540)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5181798764399224991)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 3432038368502799491)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 16660559128391391789)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/d

In [ ]:
learning_strategy(image_size=64,previous_epoch=250,epoch_size=300,batchsize=512,learning_rate=0.00006)

Loading previous model with least val loss: /gdrive/My Drive/tpuweights/vddddAssignment4Amodel_203_1.96.hdf5
Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.91.228.226:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 7344198457844153779)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 6934899072856692044)
INFO:tensorflow:*** Available Device: _DeviceAttribute

In [ ]:
learning_strategy(image_size=64,previous_epoch=300,epoch_size=325,batchsize=512,learning_rate=0.00006)

Loading previous model with least val loss: /gdrive/My Drive/tpuweights/vddddAssignment4Amodel_203_1.96.hdf5
Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.22.123.178:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 17802238759688487900)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 2403919027554651168)
INFO:tensorflow:*** Available Device: _DeviceAttribut

In [ ]:
tf.keras.backend.clear_session()
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  
modelfinal = fetch_model(64)
modelfinal.compile(loss='categorical_crossentropy',optimizer=tf.train.AdamOptimizer(learning_rate=0.00006),metrics=['accuracy'])
final_model_tpu = tf.contrib.tpu.keras_to_tpu_model(modelfinal,strategy=tf.contrib.tpu.TPUDistributionStrategy(tf.contrib.cluster_resolver.TPUClusterResolver(tpu=TPU_WORKER)))

final_validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(64,64),
                                                    color_mode='rgb', class_mode='categorical', batch_size=512, shuffle=False, seed=42)
score = final_model_tpu.evaluate_generator(final_validation_generator)
print(final_model_tpu.metrics_names)
print(score)

Loading previous model with least val loss: /gdrive/My Drive/tpuweights/vddddAssignment4Amodel_203_1.96.hdf5
INFO:tensorflow:Querying Tensorflow master (grpc://10.19.20.58:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 3188175346484654332)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 9322828699166116140)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 7441547133411068124)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 16478797923714726733)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/de

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

(x_batch,y_batch) = next(train_generator)

# Plot Generator images.
def plot_object(image):
  fig = plt.figure(figsize=(4,4))
  plt.imshow(image, interpolation="nearest")
  plt.show()

for i in range(4):
    index = random.randint(1,500)
    print(x_batch[index])
    plot_object(x_batch[index])
    print(adaptive_equalisation(x_batch[index]))
    plot_object(adaptive_equalisation(x_batch[index]))